In [220]:
import requests
import pandas as pd
import numpy as np

# Get TimeSeries

In [221]:
def getcur(curr):
    if curr=='flow':
        res = requests.get(
            'https://api.cryptowat.ch/markets/kraken/flowusd/ohlc',
        params={
        'periods': '3600',
        'after': str(int(pd.Timestamp('2021-11-22').timestamp()))
        })
    else:
        res = requests.get(
            f'https://api.cryptowat.ch/markets/coinbase-pro/{curr}usd/ohlc',
        params={
        'periods': '3600',
        'after': str(int(pd.Timestamp('2021-11-22').timestamp()))
        })
    df=pd.DataFrame(res.json()['result']['3600'],columns=['ts', 'open', 'high', 'low', 'close', 'volume', 'volumeUSD'])
    df['chain']=curr
    return df

In [248]:
currencies=['eth','sol','avax','usdt','flow']
df_list=[]
for curr in currencies:
    df=getcur(curr)
    df_list.append(df)
    

In [250]:
df_long=pd.concat(df_list)

In [251]:
df_long

,ts,open,high,low,close,volume,volumeUSD,chain
0,1637539200,4317.980,4342.240,4246.070,4262.990,7262.562789,3.123077e+07,eth
1,1637542800,4263.040,4270.340,4212.450,4234.370,8437.946084,3.574586e+07,eth
2,1637546400,4234.870,4246.720,4171.170,4217.890,9259.725370,3.895639e+07,eth
3,1637550000,4217.880,4223.480,4163.580,4193.470,9259.899519,3.882032e+07,eth
4,1637553600,4192.950,4213.590,4147.000,4168.350,7934.546906,3.315476e+07,eth
...,...,...,...,...,...,...,...,...
518,1639404000,9.131,9.131,8.995,9.035,33930.957700,3.076169e+05,flow
519,1639407600,9.034,9.036,8.851,8.892,21271.339724,1.896291e+05,flow
520,1639411200,8.892,8.892,8.730,8.731,11413.457691,1.009434e+05,flow
521,1639414800,8.768,8.808,8.695,8.769,52918.874075,4.625845e+05,flow


# Get Total USD Volume Traded

In [225]:
vols=[]
for curr in currencies:
    vols.append(df_long[df_long['chain']==curr]['volumeUSD'].sum())
total_vols=pd.DataFrame([vols,currencies]).T
total_vols.rename(columns={0:'Volume (USD)',1:'Currency'},inplace=True)
total_vols.set_index('Currency',inplace=True)
total_vols.sort_values('Volume (USD)',ascending=False)

,Volume (USD)
Currency,
eth,20265042688.186661
sol,6610358125.018849
avax,3336805161.19944
usdt,1919872322.480056
flow,84187496.788246


# Get ETH/Sol Ratio


In [226]:
df_ratio=pd.DataFrame(df_long[df_long['chain']=='eth'].set_index('ts')['close']/df_long[df_long['chain']=='sol'].set_index('ts')['close'])
df_ratio.index=pd.to_datetime(df_ratio.index,unit='s')
df_ratio.rename(columns={'close':'ETH/SOL Ratio'},inplace=True)
df_ratio

,ETH/SOL Ratio
ts,
2021-11-22 00:00:00,18.471694
2021-11-22 01:00:00,18.736150
2021-11-22 02:00:00,18.577985
2021-11-22 03:00:00,18.794937
2021-11-22 04:00:00,18.891059
...,...
2021-12-13 13:00:00,23.959647
2021-12-13 14:00:00,24.186603
2021-12-13 15:00:00,24.126092


# Change Name of Volume

In [233]:
df_long.rename(columns={'volume':'volumeBase','volumeUSD':'volumeTerm'},inplace=True) 
df_long

,ts,open,high,low,close,volumeBase,volumeTerm,chain
0,1637539200,4317.980,4342.240,4246.070,4262.990,7262.562789,3.123077e+07,eth
1,1637542800,4263.040,4270.340,4212.450,4234.370,8437.946084,3.574586e+07,eth
2,1637546400,4234.870,4246.720,4171.170,4217.890,9259.725370,3.895639e+07,eth
3,1637550000,4217.880,4223.480,4163.580,4193.470,9259.899519,3.882032e+07,eth
4,1637553600,4192.950,4213.590,4147.000,4168.350,7934.546906,3.315476e+07,eth
...,...,...,...,...,...,...,...,...
517,1639400400,9.126,9.299,9.123,9.130,36156.486241,3.323358e+05,flow
518,1639404000,9.131,9.131,8.995,9.035,33930.957700,3.076169e+05,flow
519,1639407600,9.034,9.036,8.851,8.892,21271.339724,1.896291e+05,flow
520,1639411200,8.892,8.892,8.730,8.731,11413.457691,1.009434e+05,flow


# Fat Table

In [229]:
df_fat=df_long.pivot_table(index='ts',values='close',columns='chain')
df_fat.index=pd.to_datetime(df_fat.index,unit='s')

In [247]:
df_fat

chain,avax,eth,flow,sol,usdt
ts,,,,,
2021-11-22 00:00:00,128.67,4262.99,14.150,230.785,1.0012
2021-11-22 01:00:00,134.85,4234.37,13.621,226.000,1.0013
2021-11-22 02:00:00,132.54,4217.89,13.753,227.037,1.0012
2021-11-22 03:00:00,133.38,4193.47,13.652,223.117,1.0011
2021-11-22 04:00:00,136.29,4168.35,13.581,220.652,1.0009
...,...,...,...,...,...
2021-12-13 13:00:00,83.63,3990.00,9.130,166.530,1.0006
2021-12-13 14:00:00,83.55,3975.31,9.035,164.360,1.0005
2021-12-13 15:00:00,81.57,3920.49,8.892,162.500,1.0003


# Log Return

In [245]:
df_return=np.log(df_fat/df_fat.shift())
df_return

chain,avax,eth,flow,sol,usdt
ts,,,,,
2021-11-22 00:00:00,NaN,NaN,NaN,NaN,NaN
2021-11-22 01:00:00,0.046912,-0.006736,-0.038102,-0.020952,0.0001
2021-11-22 02:00:00,-0.017279,-0.003900,0.009644,0.004578,-0.0001
2021-11-22 03:00:00,0.006318,-0.005806,-0.007371,-0.017417,-0.0001
2021-11-22 04:00:00,0.021583,-0.006008,-0.005214,-0.011109,-0.0002
...,...,...,...,...,...
2021-12-13 13:00:00,-0.010113,-0.006049,0.000657,-0.007836,0.0002
2021-12-13 14:00:00,-0.000957,-0.003688,-0.010460,-0.013116,-0.0001
2021-12-13 15:00:00,-0.023984,-0.013886,-0.015954,-0.011381,-0.0002


# Correlation Table

In [246]:
df_return.corr()

chain,avax,eth,flow,sol,usdt
chain,,,,,
avax,1.000000,0.643128,0.387489,0.621645,-0.016733
eth,0.643128,1.000000,0.560471,0.737293,-0.116483
flow,0.387489,0.560471,1.000000,0.444337,-0.196114
sol,0.621645,0.737293,0.444337,1.000000,0.110018
usdt,-0.016733,-0.116483,-0.196114,0.110018,1.000000
